# Linear Models

Check the performance of SVM and logistic regression on our engineered data sets before moving to more complex models.

In [ ]:
%reload_ext autoreload
%autoreload 2
import sys, os
import pandas as pd
import warnings
import numpy as np
from sklearn.model_selection import train_test_split
from pathlib import Path

warnings.filterwarnings('ignore')
sys.path.append(os.path.abspath(".."))
DATA_DIR = Path('../data')


from src.utils import log_metric, set_csv_path

set_csv_path(DATA_DIR / 'metrics.csv')

